# LIBRERÍAS

In [ ]:
import numpy as np
import pandas as pd
from math import sqrt
from scipy.stats import t
from sklearn import metrics
import statsmodels.api as sm
from numpy import average, std
import seaborn as sns;sns.set()
import matplotlib.pyplot as plt
from sklearn import linear_model
from pandas.core import datetools
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist
from sklearn.cluster import SpectralClustering
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets.samples_generator import make_blobs

# IMPORTAR DATOS

MATRIZ

In [ ]:
data = pd.ExcelFile('SECOP (PAE) (FINAL).xlsx')
matriz = pd.read_excel(data,'MATRIZ',names=["ID","PLAZO (CLASE)","VALOR (CLASE)","A. TIEMPO (CLASE)","A. DINERO (CLASE)","PLAZO","VALOR","A. TIEMPO","A. DINERO","URL"])

W = matriz[['PLAZO (CLASE)']].values
X = matriz[['VALOR (CLASE)']].values
Y = matriz[['A. TIEMPO (CLASE)']].values
Z = matriz[['A. DINERO (CLASE)']].values

W2 = matriz[['PLAZO']].values
X2 = matriz[['VALOR']].values
Y2 = matriz[['A. TIEMPO']].values
Z2 = matriz[['A. DINERO']].values

Q = matriz[['PLAZO (CLASE)','VALOR (CLASE)','A. TIEMPO (CLASE)','A. DINERO (CLASE)']]

# ELBOW METHOD

FIT MATRIZ

In [ ]:
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(Q)
    kmeanModel.fit(Q)
    distortions.append(sum(np.min(cdist(Q, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / Q.shape[0])
 
plt.plot(K, distortions, 'bx-')
plt.xlabel('K')
plt.ylabel('Wk')
plt.title('Elbow Method')
plt.show()

# DBSCAN

In [ ]:
db = DBSCAN(eps=1.5,algorithm="brute")
db.fit_predict(Q)
pca_2 = PCA(2)
plot_columns = pca_2.fit_transform(Q)
plt.scatter(x=plot_columns[:,0], y=plot_columns[:,1], c=db.labels_,cmap="viridis")
plt.xlabel('Canonical variable 1')
plt.ylabel('Canonical variable 2')
plt.title('Scatterplot of Canonical Variables for 3 Clusters')
plt.show()

# K-MEANS

ANALYSIS

In [ ]:
kmeanModel = KMeans(n_clusters=3).fit(Q)
y_kmeans = kmeanModel.predict(Q)
pca_2 = PCA(2,whiten=True,svd_solver="full")
plot_columns = pca_2.fit_transform(Q)
plt.scatter(x=plot_columns[:,0], y=plot_columns[:,1], c=kmeanModel.labels_,cmap="viridis")
plt.xlabel('Canonical variable 1')
plt.ylabel('Canonical variable 2')
plt.title('Scatterplot of Canonical Variables for 3 Clusters')
plt.show()

# GRÁFICAS CON CLUSTERS

PLAZO - VALOR

In [ ]:
plt.plot()
plt.xlim([min(X)-0.1, max(X)+0.1])
plt.ylim([min(Y)-0.1, max(Y)+0.1])
plt.title('PLAZO vs. VALOR')
plt.xlabel('PLAZO')
plt.ylabel('VALOR')
plt.scatter(X, Y,c=kmeanModel.labels_,cmap="viridis")
plt.show()

A. TIEMPO - A. DINERO

In [ ]:
plt.plot()
plt.xlim([min(Y)-0.1, max(Y)+0.1])
plt.ylim([min(Z)-0.1, max(Z)+0.1])
plt.title('A. TIEMPO vs. A. DINERO')
plt.xlabel('A. TIEMPO')
plt.ylabel('A. DINERO')
plt.scatter(Y, Z,c=kmeanModel.labels_,cmap="viridis")
plt.show()

PLAZO - A. TIEMPO

In [ ]:
plt.plot()
plt.xlim([min(W)-0.1, max(W)+0.1])
plt.ylim([min(Y)-0.1, max(Y)+0.1])
plt.title('PLAZO vs. A. TIEMPO')
plt.xlabel('PLAZO')
plt.ylabel('A. TIEMPO')
plt.scatter(W, Y,c=kmeanModel.labels_,cmap="viridis")
plt.show()

VALOR - A. DINERO

In [ ]:
plt.plot()
plt.xlim([min(X)-0.1, max(X)+0.1])
plt.ylim([min(Z)-0.1, max(Z)+0.1])
plt.title('VALOR vs. A. DINERO')
plt.xlabel('VALOR')
plt.ylabel('A. DINERO')
plt.scatter(X, Z,c=kmeanModel.labels_,cmap="viridis")
plt.show()

# CENTROIDES

ACTUALIZAR MATRIZ

In [ ]:
matriz["CLUSTER"]=pd.DataFrame(list(y_kmeans))

CENTROIDES

In [ ]:
cluster_0=kmeanModel.cluster_centers_[0]
cluster_1=kmeanModel.cluster_centers_[1]
cluster_2=kmeanModel.cluster_centers_[2]

DITANCIA EUCLIDIANA

In [ ]:
matriz["VECTOR (CLASE)"]=matriz[['PLAZO (CLASE)','VALOR (CLASE)','A. TIEMPO (CLASE)','A. DINERO (CLASE)']].values.tolist()

In [ ]:
clusters=[]
for row in matriz["CLUSTER"]:
    if row == 0:
        clusters.append(cluster_0.tolist())
    elif row == 1:
        clusters.append(cluster_1.tolist())
    elif row ==2:
        clusters.append(cluster_2.tolist())

matriz["CENTROIDE (CLASE)"]=clusters

# ECUACIONES HISTOGRAMICAS ATRIBUYENTES

ESTIMACION DE POLINOMIOS

In [ ]:
# x = np.array([9,10,11,12])
# y = np.array([600,5000,15000,64300])
# z = np.polyfit(x, y, 6)
# list(z)

ECUACIONES

In [ ]:
#___________________PLAZO

def plazo(x):
    
    y= (31.75)*x**4 - (339.5)*x**3 + (1281.8)*x**2 - (1902)*x + 1045
    
    return abs(y)

#___________________VALOR

def valor(x):
    
    if x <= 9:
        y = (0.2039)*x**6 - (5.4724)*x**5 + (57.417)*x**4 - (297.35)*x**3 + (788.44)*x**2 - (986.32)*x + 455.67
    else:
        y = 0.5801366953438196*x**6-4.3475606700219505*x**5-69.06764332045618*x**4-201.42414196416192*x**3+6154.0731469554385*x**2+85790.86522364747*x-721596.0201049197
    return abs(y)

#___________________A. TIEMPO

def atiempo(x):
    
    y = (0.135)*x**3 - (0.8699)*x**2 + (1.8748)*x - 1.1399
    
    return abs(y)

#___________________A. DINERO

def adinero(x):
    
    y = (-0.0127)*x**4 + (0.1702)*x**3 - (0.7541)*x**2 + (1.441)*x - 0.8445
    
    return abs(y)

VECTORES CLUSTER

In [ ]:
cluster_0_v=cluster_0.copy()
cluster_1_v=cluster_1.copy()
cluster_2_v=cluster_2.copy()

for i in range(4):
    if i == 0:
        cluster_0_v[i]=plazo(cluster_0[i])
    if i == 1:
        cluster_0_v[i]=valor(cluster_0[i])
    if i == 2:
        cluster_0_v[i]=atiempo(cluster_0[i])
    if i == 3:
        cluster_0_v[i]=adinero(cluster_0[i])

for i in range(4):
    if i == 0:
        cluster_1_v[i]=plazo(cluster_1[i])
    if i == 1:
        cluster_1_v[i]=valor(cluster_1[i])
    if i == 2:
        cluster_1_v[i]=atiempo(cluster_1[i])
    if i == 3:
        cluster_1_v[i]=adinero(cluster_1[i])
        
for i in range(4):
    if i == 0:
        cluster_2_v[i]=plazo(cluster_2[i])
    if i == 1:
        cluster_2_v[i]=valor(cluster_2[i])
    if i == 2:
        cluster_2_v[i]=atiempo(cluster_2[i])
    if i == 3:
        cluster_2_v[i]=adinero(cluster_2[i])


clusters_vector=[]
for row in matriz["CLUSTER"]:
    if row == 0:
        clusters_vector.append(cluster_0_v.tolist())
    elif row == 1:
        clusters_vector.append(cluster_1_v.tolist())
    elif row ==2:
        clusters_vector.append(cluster_2_v.tolist())

matriz["CENTROIDE (VECTOR)"]=clusters_vector

DISTANCIAS

In [ ]:
distances=[]

for i in range(0,len(matriz)):
    distances.append(distance.euclidean(matriz["CENTROIDE (CLASE)"][i],matriz["VECTOR (CLASE)"][i]))

matriz["DISTANCIA"]=distances

matriz=matriz[["ID","URL","PLAZO (CLASE)","VALOR (CLASE)","A. TIEMPO (CLASE)","A. DINERO (CLASE)","VECTOR (CLASE)", "CLUSTER","CENTROIDE (CLASE)", "DISTANCIA"]]
matriz.columns = matriz.columns.get_level_values(0)

# MATRIZ

In [ ]:
matriz

# PRUEBAS ESTADISTICAS

VISTA GENERAL

In [ ]:
matriz.describe()

REGRESION LINEAL MULTIPLE

In [ ]:
B = matriz[['PLAZO (CLASE)','VALOR (CLASE)','A. TIEMPO (CLASE)','A. DINERO (CLASE)']]
Q = matriz[["DISTANCIA"]]

# MODELO
# B=sm.add_constant(B)
model = sm.OLS(Q, B).fit()

# RESUMEN
model.summary()

INTERVALO DE CONFIANZA (DISTANCIA) - GLOBAL

In [ ]:
data = matriz["DISTANCIA"].values
mean = average(data)

stddev = std(data, ddof=1)

t_bounds = t.interval(0.95, len(data) - 1)

cG = [mean + critval * stddev / sqrt(len(data)) for critval in t_bounds]

print "Media: %f" % mean
print "Tamano Muestra: ", len(data)
print "Intervalo de Confianza 95%%: [%f, %f]" % (cG[0], cG[1])

INTERVALO DE CONFIANZA (DISTANCIA) - CLUSTER 0

In [ ]:
data = matriz.loc[matriz["CLUSTER"]==0]
data = data["DISTANCIA"].values
mean = average(data)

stddev = std(data, ddof=1)

t_bounds = t.interval(0.95, len(data) - 1)

c0 = [mean + critval * stddev / sqrt(len(data)) for critval in t_bounds]

print "Media: %f" % mean
print "Tamano Muestra: ", len(data)
print "Intervalo de Confianza 95%%: [%f, %f]" % (c0[0], c0[1])

INTERVALO DE CONFIANZA (DISTANCIA) - CLUSTER 1

In [ ]:
data = matriz.loc[matriz["CLUSTER"]==1]
data = data["DISTANCIA"].values
mean = average(data)

stddev = std(data, ddof=1)

t_bounds = t.interval(0.95, len(data) - 1)

c1 = [mean + critval * stddev / sqrt(len(data)) for critval in t_bounds]

print "Media: %f" % mean
print "Tamano Muestra: ", len(data)
print "Intervalo de Confianza 95%%: [%f, %f]" % (c1[0], c1[1])

INTERVALO DE CONFIANZA (DISTANCIA) - CLUSTER 2

In [ ]:
data = matriz.loc[matriz["CLUSTER"]==2]
data = data["DISTANCIA"].values
mean = average(data)

stddev = std(data, ddof=1)

t_bounds = t.interval(0.95, len(data) - 1)

c2 = [mean + critval * stddev / sqrt(len(data)) for critval in t_bounds]

print "Media: %f" % mean
print "Tamano Muestra: ", len(data)
print "Intervalo de Confianza 95%%: [%f, %f]" % (c2[0], c2[1])

DENTRO DE RANGO GLOBAL

In [ ]:
matriz["IN-RANGO GLOBAL"]=np.where(matriz["DISTANCIA"]<=cG[1],0,1)

DENTRO DE RANGO LOCAL

In [ ]:
matriz["IN-RANGO LOCAL"]=np.where((matriz["CLUSTER"]==0)&(matriz["DISTANCIA"]<=c0[1]),0,
                                 np.where((matriz["CLUSTER"]==1)&(matriz["DISTANCIA"]<=c1[1]),0,
                                          np.where((matriz["CLUSTER"]==2)&(matriz["DISTANCIA"]<=c1[1]),0,1)))

DENTRO DE RANGOS GLOBAL Y LOCAL

In [ ]:
matriz["CALIFICACION"]=np.where(matriz["IN-RANGO LOCAL"]==0,0,1)

VISTA PRELIMINAR

In [ ]:
print "Número de nodos en rango global: ", len(matriz.loc[matriz["IN-RANGO GLOBAL"]==0]),"(",float(len(matriz.loc[matriz["IN-RANGO GLOBAL"]==0]))/float(len(matriz))*100,"%",")"
print ""
print "Número de nodos en rango local (0): ", len(matriz.loc[(matriz["IN-RANGO LOCAL"]==0)&(matriz["CLUSTER"]==0)]),"(",float(float(len(matriz.loc[(matriz["IN-RANGO LOCAL"]==0)&(matriz["CLUSTER"]==0)]))/float(len(matriz.loc[matriz["CLUSTER"]==0])))*100,"%",")"
print "Número de nodos en rango local (1): ", len(matriz.loc[(matriz["IN-RANGO LOCAL"]==0)&(matriz["CLUSTER"]==1)]),"(",float(float(len(matriz.loc[(matriz["IN-RANGO LOCAL"]==0)&(matriz["CLUSTER"]==1)]))/float(len(matriz.loc[matriz["CLUSTER"]==1])))*100,"%",")"
print "Número de nodos en rango local (2): ", len(matriz.loc[(matriz["IN-RANGO LOCAL"]==0)&(matriz["CLUSTER"]==2)]),"(",float(float(len(matriz.loc[(matriz["IN-RANGO LOCAL"]==0)&(matriz["CLUSTER"]==2)]))/float(len(matriz.loc[matriz["CLUSTER"]==2])))*100,"%",")"
print ""
print "Cluster 0 (#): ", len(matriz.loc[matriz["CLUSTER"]==0])
print "Cluster 1 (#): ", len(matriz.loc[matriz["CLUSTER"]==1])
print "Cluster 2 (#): ", len(matriz.loc[matriz["CLUSTER"]==2])

In [ ]:
matriz.describe()

REGRESION LOGISTICA

In [ ]:
dummy_cluster = pd.get_dummies(matriz['CLUSTER'], prefix='CLUSTER')
cols_to_keep = ['PLAZO (CLASE)','VALOR (CLASE)','A. TIEMPO (CLASE)','A. DINERO (CLASE)']
data = matriz[cols_to_keep].join(dummy_cluster)
data['INTERCEPT'] = 1.0

REG LOG CLUSTER 0

In [ ]:
data0=data.copy()
data0=data0[["CLUSTER_0","PLAZO (CLASE)","VALOR (CLASE)","A. TIEMPO (CLASE)","A. DINERO (CLASE)"]]
data0.columns = data0.columns.get_level_values(0)

train_cols = data0.columns[1:]

logit = sm.Logit(data0['CLUSTER_0'], data0[train_cols])

result = logit.fit()
result.summary()

In [ ]:
data1=data.copy()
data1=data1[["CLUSTER_1","PLAZO (CLASE)","VALOR (CLASE)","A. TIEMPO (CLASE)","A. DINERO (CLASE)"]]
data1.columns = data1.columns.get_level_values(0)

train_cols = data1.columns[1:]

logit = sm.Logit(data1['CLUSTER_1'], data1[train_cols])

result = logit.fit()
result.summary()

In [ ]:
data2=data.copy()
data2=data2[["CLUSTER_2","PLAZO (CLASE)","VALOR (CLASE)","A. TIEMPO (CLASE)","A. DINERO (CLASE)"]]
data2.columns = data2.columns.get_level_values(0)

train_cols = data2.columns[1:]

logit = sm.Logit(data2['CLUSTER_2'], data2[train_cols])

result = logit.fit()
result.summary()

# MATRIZ

In [ ]:
matriz

# IMPRIMIR MATRIZ

writer = pd.ExcelWriter('MATRIZ_FINAL.xlsx')
matriz.to_excel(writer,'MATRIZ',index=False)
writer.save()